## Agregamos ahora otra funcion objetivo para que cada conjunto tenga un tamaño similar

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import math
from utils import *
import dimod
import warnings
import matplotlib.pyplot as plt
from functions_dwave import *
warnings.filterwarnings('ignore')

In [ ]:
# Creamos una instancia del problema
set = np.array([3, 1, 1, 2, 2, 1])
set = np.array([7, 15, 3, 2, 5, 5, 10, 7])
suma = sum(set)
size = len(set)
data = dict({'set': set})
df = pd.DataFrame(data)
print("-------------------------------------")
print("Partition the set: \n ")
print(df)
print("With total sum of: ", suma)

In [ ]:
# q_sum hace que la suma de los dos conjuntos sea similar. q_size que los conjuntos tengan dimensiones similares
q_sum = np.outer(set, set) - suma * np.diag(set)
q_size = np.ones((size, size)) - size * np.identity(size)

alpha = 1
beta = 15
qubo = alpha * q_sum + beta * q_size
print("Matriz objetivo:\n")
print(qubo)

In [ ]:
# Enviamos problema a Dwave:
print("simmulating....")
sampleset = sendToDwave(qubo, 100)
print("Filtering:")
# Nos quedamos con la solucion que minimiza la energia:
energies = [element[1] for element in sampleset]
solution = sampleset[energies.index(min(energies))][0] # Le saco la energia
print(solution)


plt.bar([(str(el[1])) for el in sampleset], [(el[1]) for el in sampleset])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
index_of_chosen = np.where(solution == 1)[0] # retorna los indices de los items elegidos.
output_df = df.iloc[index_of_chosen]
set2 = np.ones(len(set)) - solution
index_of_chosen_set2 = np.where(set2 == 1)[0]
output_df_set2 = df.iloc[index_of_chosen_set2]
print("-------------------------------------")
print("Choosen items are: ")
print(output_df)
print("-------------------------------------")
print("-------------------------------------")
print("Choosen items in Set 2 are: ")
print(output_df_set2)
print("-------------------------------------")

sum_set_1 = np.dot(set, solution)
sum_set_2 = suma - sum_set_1
print("Suma del primer conjunto = ", sum_set_1)
print("Suma del segundo conjunto = ", sum_set_2)